In [1]:
import os
from io import StringIO

import sys

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model', 'CFD')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'RD-systems-and-test-benches')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss', 'ansys')))

import numpy as np
import pandas as pd
import openpyxl

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import jou_gen as jg
import bridge_module as bm

import ansys.fluent.core as pyfluent

from CoolProp.CoolProp import PropsSI

import hx_hydraulic as hxhy

from tqdm import tqdm

In [2]:
computer = 'seagull'
jg.set_computer(computer)
server_number = 3000

category_folder_name = 'Conduction'
subcategory_folder_name = ''
project_folder_name = 'fin_tube'

caoMeshFolder = 'L20mm'
title = 'parametric_try0'

# report_names = ['report_outlet_sp', 'report_inlet_sp', 'report_outlet_mdot']

In [ ]:
if computer == 'seagull':
    root_fp = r'D:\ANSYS Fluent Projects'
else:
    root_fp = "/usrtmp/delachaux/ANSYS Fluent Projects"
    jg.computer = computer

main_fp = os.path.join(root_fp, category_folder_name, subcategory_folder_name, project_folder_name)
Mesh_fp = os.path.join(main_fp, 'mesh')
SR_fp = bm.check_folder(os.path.join(main_fp, 'SimulationResults'))
caoMesh_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder))
param_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder, title))

tui, solver = bm.init_solver(main_fp, str(server_number), subfolder='')
print('main_fp :',main_fp)
print('param_fp :',param_fp)

In [4]:
nb_it = 500

cellZoneToDelete_tube_list = ["", "d15", "d20"]
tube_bc_list = ["tube10", "tube15", "tube20-shadow"]
cellZoneToDelete_extension_list = ["", "ext25", "ext20", "ext15"]
free_end_bc_list = ["free_end25",  "free_end20-shadow", "free_end15-shadow", "free_end10-shadow"]

Dtube_list = [0.010,0.015,0.020]
Hext_list = [0.025,0.020,0.015,0.010]

cellZoneToDelete_tube_list = cellZoneToDelete_tube_list[0:2]
tube_bc_list = tube_bc_list[0:2]
cellZoneToDelete_extension_list = cellZoneToDelete_extension_list[2:4]
free_end_bc_list = free_end_bc_list[2:4]
Dtube_list = Dtube_list[0:2]
Hext_list = Hext_list[2:4]

h_range = np.array([1,3,5,7])
T_amb = 273.15 + 0.
T_tube_range = np.arange(T_amb-16, T_amb, 4.)
res_list = []

In [5]:
nb_it = 5000

res_list = []

In [ ]:
L_list = [60]

for g, L in enumerate(L_list):

    caseName = fr""" "D:\ANSYS Fluent Projects\Conduction\fin_tube\fluent\L{L}mm.cas.h5" """
    print('L', L, ' mm')

    for i, val in tqdm(enumerate(cellZoneToDelete_tube_list), total=len(cellZoneToDelete_tube_list)):

        if i == 0:
            pass
        
        else:

            print('D', round(Dtube_list[i]*1000,1), ' mm')
            string_list = [f"""/file/read-case {caseName}"""]
            jg.save_journal(tui, string_list, "read_case_reset", read=True)

            if val == "":
                pass
            else:
                for ii, cellZoneToDelete_tube in enumerate(cellZoneToDelete_tube_list[1:i+1]):
                    string_list = [f"""/define/boundary-conditions/modify-zones/delete-cell-zone {cellZoneToDelete_tube}  """]
                    jg.save_journal(tui, string_list, "delete_cell_zone", read=True)

            tube_bc = tube_bc_list[i]
            string_list = [f"""/define/boundary-conditions/wall {tube_bc} 0 no 0 no yes temperature no "T_tube" no no 1"""]
            jg.save_journal(tui, string_list, "set_free_end", read=True)

            for j, cellZoneToDelete_extension in enumerate(cellZoneToDelete_extension_list):

                if cellZoneToDelete_extension == "":
                    pass
                else:
                    string_list = [f"""/define/boundary-conditions/modify-zones/delete-cell-zone {cellZoneToDelete_extension}  """]
                    jg.save_journal(tui, string_list, "delete_cell_zone", read=True)

                free_end_bc = free_end_bc_list[j]
                string_list = [f"""/define/boundary-conditions/wall {free_end_bc} 0 no 0 no yes convection no "h" no "T_amb" no no 1"""]
                jg.save_journal(tui, string_list, "set_free_end", read=True)

                for k, T_tube in enumerate(T_tube_range):
                    jg.change_named_expression(tui, 'T_tube', T_tube, 'K')
                    for l, h in enumerate(h_range):
                        jg.change_named_expression(tui, 'h', h, 'kg s^-3 K^-1')

                        solver.solution.initialization.hybrid_initialize()

                        if computer == 'seagull':
                            solver.solution.run_calculation.iterate(number_of_iterations = nb_it)
                        elif computer == 'lmps_cds':
                            solver.solution.run_calculation.iterate(iter_count = nb_it)

                        jg.write_report(tui,"ht",param_fp,"report_all_ht")
                        df = pd.read_csv(os.path.join(param_fp, 'report_all_ht.csv'), sep=',')
                        res_list.append({
                                        'L' : L,
                                        'Dtube' : Dtube_list[i],
                                        'height' : Hext_list[j],
                                        'T_tube' : T_tube,
                                        'h' : h, 
                                        'transfer_to_tube' : - df.loc[df['Component'] == tube_bc, 'ht'].values[0]
                                        }
                                        )

In [7]:
res_df = pd.DataFrame(res_list)

In [10]:
import pickle

In [28]:
with open(os.path.join(param_fp, 'res_df_L20.pkl'), 'wb') as f:
    pickle.dump(df20, f)

In [24]:
with open(os.path.join(param_fp, 'res_df.pkl'), 'rb') as f:
    df = pickle.load(f)

In [26]:
df20 = df.loc[df['L'] == 20]

In [ ]:
df20

In [ ]:
res_df['Dtube'].unique()

### Plot

In [ ]:
res_df40 = res_df.loc[res_df['L'] == 60]

res_df40 = res_df40.loc[(res_df40['Dtube'] == 0.01) & (res_df40['height'] == 0.015)]

for T_tube in T_tube_range:
    locat = res_df40.loc[res_df40['T_tube'] == T_tube]
    plt.plot(locat['h'], locat['transfer_to_tube'], label = f'T_tube = {T_tube}')

plt.legend()

In [ ]:
for h in h_range:
    locat = res_df.loc[res_df['h'] == h]
    plt.plot(locat['T_tube'], locat['transfer_to_tube'], label = f'h = {h}')

plt.legend()